In [ ]:

# mount drive https://datascience.stackexchange.com/questions/29480/uploading-images-folder-from-my-system-into-google-colab
# login with your google account and type authorization code to mount on your google drive.
import os
from google.colab import drive
drive.mount('/gdrive')




Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
!fusermount -u drive
!google-drive-ocamlfuse drive

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from pandas.core.common import flatten
import copy
import numpy as np
import random

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

import glob
from tqdm import tqdm

fusermount: failed to unmount /content/drive: Invalid argument
/bin/bash: google-drive-ocamlfuse: command not found


In [ ]:
# do not run below

In [ ]:

import torch.nn as nn
import torchvision.models as models

class inception(nn.Module):
    def __init__(self, train_inc=False, num_classes=3):
        super(inception, self).__init__()
        self.train_inc = train_inc
        self.inception = models.inception_v3(pretrained=True, aux_logits=True)
        self.inception.dropout=nn.Dropout(0.3)
        self.inception.fc = nn.Linear(self.inception.fc.in_features, num_classes)
        
        self.dropout = nn.Dropout(0.2)
        self.soft = nn.Softmax()

    def forward(self, images):
        features = self.inception(images)
        features_2 = self.dropout(features)
        return self.soft(features_2).squeeze(1)

In [ ]:
print(model)

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

from tqdm import tqdm
device = ("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
transform = transforms.Compose(
        [
            transforms.Resize((356, 356)),
            transforms.RandomCrop((299, 299)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    )

In [ ]:
!pip install kora -q
from kora import drive
drive.link_nbs()





     |████████████████████████████████| 57 kB 950 kB/s 
     |████████████████████████████████| 1.6 MB 37.5 MB/s 
Mounted at /content/drive


In [ ]:
device = ("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
import location_dataset
# from location_dataset import *
# importing wasnt really useful

In [ ]:
from torch.utils.data import Dataset
import pandas as pd
import os
from PIL import Image
import torch

class Damage_location(Dataset):
    def __init__(self, root_dir_front,root_dir_side, root_dir_rear, annotation_file, transform=None):
        self.root_dir1 = root_dir_front
        self.root_dir2 = root_dir_side
        self.root_dir3 = root_dir_rear
        self.annotations = pd.read_csv('/gdrive/MyDrive/project/' +annotation_file)
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_id = self.annotations.iloc[index, 0]
        value = self.annotations.iloc[index, 1]
        if value ==0:
          root_dir = self.root_dir1
        elif value == 1:
          root_dir = self.root_dir2
        elif value ==2:
          root_dir = self.root_dir3
        img = Image.open(os.path.join(root_dir, img_id)).convert("RGB")
        y_label = torch.tensor(float(self.annotations.iloc[index, 1]))

        if self.transform is not None:
            img = self.transform(img)

        return (img, y_label)

In [ ]:
root2= '/gdrive/MyDrive/project/car-damage-dataset/data2a'
training = root2+'/training'
# we then split the training in train, valid so we store everything in the training folder
front = '/00-front/'
rear = '/01-rear/'
side = '/02-side/'


In [ ]:
%store -r learning_rate
num_epochs = 40
learning_rate = 0.0001
train_CNN = False
batch_size = 32
shuffle = True
pin_memory = True
num_workers = 1


In [ ]:
dataset = Damage_location(training+front,training+side,training+rear,"damage_locations.csv",transform=transform)
print(dataset)

In [ ]:
dataset = Damage_location(training+front,training+side,training+rear,"damage_locations.csv",transform=transform)
train_set, validation_set = torch.utils.data.random_split(dataset,[1000,150])
train_loader = DataLoader(dataset=train_set, shuffle=shuffle, batch_size=batch_size,num_workers=num_workers,pin_memory=pin_memory)
validation_loader = DataLoader(dataset=validation_set, shuffle=shuffle, batch_size=batch_size,num_workers=num_workers, pin_memory=pin_memory)

In [ ]:
model = inception().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for name, param in model.inception.named_parameters():
    if "fc.weight" in name or "fc.bias" in name:
        param.requires_grad = True
    else:
        param.requires_grad = train_CNN

In [ ]:
max_accuracy = 1
# i do not want to update my weights now

In [ ]:
def check_accuracy(loader, model):
    if loader == train_loader:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on validation data")

    num_correct = 0
    num_samples = 0
    model.eval()
    root = '/gdrive/MyDrive/project/car_damage_location/'
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            # print(scores)

            # predictions = torch.tensor([1.0 if i >= 0.5 else 0.0 for i in scores]).to(device)
            
            predictions = torch.tensor([torch.argmax(score) for score in scores]).to(device)
            print(predictions)

            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
            ratio = float(num_correct)/float(num_samples)
            if loader != train_loader:
              global max_accuracy
              if ratio>= max_accuracy:
                max_accuracy = ratio
                print('max accuracy is:', max_accuracy)
                torch.save(model.state_dict(), root+'model_location_weights.pth')
    return f"{float(num_correct)/float(num_samples)*100:.2f}"
    print(f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}")
    model.train()

In [ ]:
def train():
    model.train()
    for epoch in range(num_epochs):
        loop = tqdm(train_loader, total = len(train_loader), leave = True)
        if epoch % 5 == 0:
            loop.set_postfix(val_acc = check_accuracy(validation_loader, model))
        for imgs, labels in loop:
            imgs = imgs.to(device)
            labels = labels.to(device)
            labels=labels.to(torch.int64)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loop.set_description(f"Epoch [{epoch}/{num_epochs}]")
            loop.set_postfix(loss = loss.item())


In [ ]:
if __name__ == "__main__":
    train()

In [ ]:
train()

In [ ]:
img =Image.open('/gdrive/MyDrive/project/car-damage-dataset/data2a/training/02-side/0257.jpeg')

In [ ]:
img.show()

In [ ]:
def preprocess(img):
  img = transform(img)

  return img


In [ ]:
# model_loaded = inception()
# path ='/gdrive/MyDrive/project/car_damage_location/model_location_weights.pth'
# model_loaded.load_state_dict(torch.load(path), map_location=torch.device('cpu'))



In [ ]:
# test
import urllib

from urllib.request import Request, urlretrieve  # Python 3
model_loaded=model

def test(image_path):
    
    urlretrieve(image_path, 'save.jpg') # or other way to upload image
    # Image.open('save.jpg')
    img = transform(Image.open('save.jpg'))
    img = img.to(device)
    img = img.unsqueeze(0)


    with torch.no_grad():
      output = model_loaded(img)
      # print(output)
      if output[0].argmax()==0:
        return ' Front damage '.format(output[0] * 100)
      elif output[0].argmax()==1:
        return ' Side damage '.format((1-output[0]) * 100)
      else:
        return ' Rear damage '.format((1-output[0]) * 100)






In [ ]:
url='https://res.cloudinary.com/debi2p5ek/image/upload/w_1140/v1/1984/37086-sideswiped_car_largeoriginal-6f27d022.png'
# url2 = 'data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBxAQEA8PEBAPDw8NDw0PDQ0PDw8NDw0PFREWFhURFRUYHSggGBolHRUVITEhJSkrLi4uFx8zODMtNygtLisBCgoKDg0OFRAQFS0dFx8tLS0tKy0tNy0rLS0tLS0rLi0rKy0tLS0tKy4tKystKy0rMSsrKy0rKy0tLS0rLS0tLf/AABEIAJYBTwMBIgACEQEDEQH/xAAcAAABBQEBAQAAAAAAAAAAAAAEAAECAwUGBwj/xAA9EAACAgEBBQYDBgQEBwEAAAABAgADBBEFEiExYQYTIkFRcQeBkRQjMkKhsTNScoJjweHwNGKSorLC8ST/xAAaAQADAQEBAQAAAAAAAAAAAAAAAQIDBAUG/8QAIhEBAQEBAAICAQUBAAAAAAAAAAECEQMhEjEEIkFRYfAU/9oADAMBAAIRAxEAPwDuMczQoMyqGmhS05nfWnUYSsBpaGVmUirhHkRJQRSjR5EmAS1jGR3ot6A4Gy69RMHKxVbvKnH3d6lH6a8j8jOisMys6vWKtcXjxPPwWotsqbnWxX36xkE7Dt3galMgD8Q3Lf6hyP0nKKkxsenjXZKkkvWVKJaoiaJaRiJMCIiIB2WR3YQVkdyMka4UhlAWXVwOLI2kkBJaQCrSLSWaSDQTXLbW/iN7zMYzS2z/ABGmW00jHRaxExoxMaSMg0cyDRpqDmFbCymS9EA1Fzom6ToN7eG63uDA3M3/AIbYve7VxAVDrWz2up4ghUOn6kS4w8l5HrGLdvDjwZeDD0MsdAeYmntTZ4LO9Q0ZdNV/nT1/eZQaaPNs4AzcDXinAj0gtObZWdLBqPWbcruoVhxEnh9/lVTlK44ESZmVk7NZDvVn5eUbH2kR4bBofXyh0c/hpsAZn5Oz1biNVPqOEPRww1B1jMIya1DQ6l5kU2Q6qyYvS1GvTZDanmPVZDarY2djTVpPegSWy0Wxo4vLStmlZtlL2wORcbJHvYK9spa6JXBr2wLJeVtdKLbIKmWdtWgW12Vn8w8PRhynnrUkEg8wSDPRLz5zmO0GHu2BwPDaN75+cz07PFeemGEkwkvFckK5DoUbsfdl+5GKQNQVjbsvKRisCVBZJRJaRCMk1EnpGSTiPqBEqcS9pTZAnJbaP3rTKYzT22fvWmW00jHf2UUaKUgjK2kzK3jiaqsM9H+B2z967JySOFarUp6ni3/rPNLWnvnwp2Z9n2bUSNHyCbW9eJ1H6aS45PLp1Vh8aDgNdQfUjnpOfvp04jkWce2hM1qUNuUgHKsFj7ngP01MB0/j1czW7HX+6bZzLL/Li3foDG3pMrISAWsHycRH5gS8iNEGHdRZUdU4r6S/H2kCPF4T1mowBgOVs9W5cDFzn0rsv2LR4XTbM1WlqWTF61jZqthVd0xa7oSl0bO4bCXy0XzJW6WC6NPwaRukGugXexjZATIhrJU1kqLxtYlTKReMYwEmqwUqavWBbXw9+huHiqO8PbzmwtcurxwdQeTAg/OHB8+POlrku7mhk4m47L/KSJUa5nx1zQQpIlIWa5Bq4cV0KVkCsKKSpliAZhGlriVGASQy0GUCWgwCTQeyXEyi0wDkNsn71veZhh+1TrY/vADNIw39mjxoo0GMqcyxjKLWlRG6s2bhnIyKaBxN1iqf6fP9NZ9LLu01Ig4KihR7AcZ4z8INmd5lvksPDQu6p/52/wBP3nsGH/8Aoyd0ca6NGf0L/lX6jX5dZpI4PJr20+z9e677w+8Yd4//AC68FH0Ep7Q1JWe8AA3lcOR5nnqZriju21I/ikAt6HTgJm9orglTbwBBDA6+XCb5z7jnuuxy6OCAfUaiJxM3s3lC3HU/ylkP9rETSPCZGbdjFY4MRMQVmVsZa0qaBAi0dXlDNGDzme7wary9LZnq8uR4C5aSWy1bJno8vR403I4PJBoMhlyRpsXAyYEatYRWkaeoKkuSuWJVCEqgi6V11wumuJK4Qixotc1t/C+83tPxDX5zGbHnabVp3lB9Jg248mxt49+mI1Mqeqa1lMHsqk8bzTKdJQ6TRtrg1iRNJQDrB3EOsWC2CJSkGSBkDFrAlhMotbnJM8FyLNFPsYByec2rsepghhF51J9zBzNIw0aIxoxMaEXMEuaXWtDuyWzTk5da6aqpDv6aDlLzHN5dvVOxWH9h2ep01uuOoUc2sbkJ6T2X2UaahvcXJ37G/msPP5Dl7Cc32fwe+tFumtWMCtI8t7k1ny5D3M9ATRV6Ac5vzjg1rt4a1wNNeJHEdJwHxOzDVjDyDkgHrO1tfgWPDzM8L+LPalrmNII3KCyJu/mc8yfYcPnL58c2lj9Wou+GWVv02r6WMw9iZ2bCec/CyzQuv8yz0czBWvuqSJEy0ythAkCZBjJmQMAxWaMGkWMiDOV9BIvV5ejwRTLkMD4MraEVmB1mGVCNFguuF1LB6FmhQkqMdVZTXDaqpGiuH1JKY6qNdMuWqWKssAj4yulQSS0k9IxEaeqrl1BEybqpsNAr1irTNY9tcEtrmpcIDdJsb5rMuSA2iaN5mde0it80HbBLYRa8CtsktYrcypnkLbYLZdAL3sgebd4G9pB75n5+RqNI01nPKGk3aVM0uMaRlbtHJg9rypGW9ciq1p6J2B2U4rCoNL8vXxedVQ5t/kOpnF9m9mNl5VVKjXfdQfTn5z6G7F9nPsqvkXgFgd2tU4ghTooHTz9/ab4y87y+R0OLipi0JSOB3RvnoPKX4zeA6kks2oHkogv2Gy7fsL+LQlF/LveQ9uAEr2rnjGoa5wAUQagct7T9p08nOfu5O23rnPiR2lGPT3KH72zhz5T552tlmx+eoXUA+p8zNvtbt58ix7GJ3rCd0fyp6/OcsJj5Nd9T6jrzn45/uvQPhy2646gz0xLZ5p2EXRl9jO/Vpmi/Y3WRaVrZJb0RItIMJNpXrAMJpGV03BgCJZOV9FEhLklSy6uAoqoQ6hYHSIfRHGWh2Os0qFgFE0aDLjn0NpEMrEEpMMQyoxq1ZOQBktY2VPImItKntEBIdzAr2iuyRM7IyhE1zkr3mZkXSOVmdZj5Wb1k2ujOV2TkTLyMiD5ObMvJzesi10SCr8iA3ZMAvzOsBtyolNC7KgdmTAbMiDvfDhWjLMiBXXayl7ZS1kuRlraxnlZaVPaBzMGfK9Br1MuZc+/LJ90WzwWxtZQzk8z8pXNJlyb83fp6z8Hdid67W6Me7Yb+mqnoAZ7o+rhUC7igABR+UTyv4Bba+4vx2C61lXWzTxFST4T66eXvPWjeBxHn5zpz6k5HDu21JW3BpwBA49J5D8Xu2GgfAqZSngbKsHE68xSP0J6Te+JXbJcaoU49iNk3bwKg6mkDm7+gnz7tTNNrHiSASd483Y82MWr8Z/da+LHf1X6ga+0uxY8z+nSRrHESMvw01dR149B5mYNrXovYurTQ9J2O9OT7CObFst5Vhu7qHqBzb6zqiYMquDSStKA0mDEF+9GIlQaOHgHE4tpU9PMTVrs1mNXDKH0nI+jrTQwiswGt4VU0aa0KTDqDMypoZTZKjPTWpaHU2TGruhCZEpjY3arYWl4nL5e166K3ttcJXWNWY/tp5npOaxNuZ20ra66LU2bRkFvs7uFszcsAElq0J0VdAeJ05cCTwlT2x1JPdeo/aBItmCeZdpeyC49KXWX7VyHs13mfNFQrYc1ICH3HPXj6TF2dszaJoGTh5eSFLWAUZNgykbcZV/FoPxEkABdfCeXOVc1nNY/l67dtAesAv2l1nnWzO11juaMlO4yF5r+Szqp/+9DD7tqdZna6c+OX26XI2l1mXlbT6znr9pdZm5G0esnraeNuZW0usyMnaPWY9+d1gF2VEvkjTvzusAuy+sAfIg72w4V3ILsyIO90HayVGyVMsteQQ1sqayD2Wgc4O+QTy4dZcy59+eQVZcBzMGfJJ5fWUGNNJmOTfm1r69HJ158Yot314fvFven185TFID5e8QUHzEhGgHrnwX8Ay3J0ARFJPIakn/KE9qfiE7u2Ph2FK0BRr+e8fMLPO9mbXsqxLMaslBe4e+wc9wDQKJVhtqdeQUaKPSXPJZJDnjn3UNp3PqxZyzWHV2J1Yj01mbCdoPq3tBhM7erOJv7MwtzHsvYcbFK1DoeH6mZ2ycE3WKn5ebn0WdatYuysfGUeCv71x5BV/CPrAtV1nZzB+z41NXmFBbqx4maJMYekYwQcmOGkIxgF29ETKgZLWIOMQwitoIrS5HnI+i60KnhVdkzK7JelsZVq12whLpjrfJjJlJsbYyYzZcxWy5RZm9YJ4lk1nPzqcZ9TjYy9/cmugus1AVGPkNXrBPo5ladnrjk15gzdWFldrMtLUWUqK67EWtSSBp3laheQB89CJmW5zJTtI1sy3d1XYrIxR91bsYtoRx/CjfQyfYHbN+ZZbi2WPdc/cvjixyx3U7wOu8f61P8Ab0nZ+LnOtZmryd9vK/O3qd+M9yensPZjald4sx7VV2OrKH8e+p5jj5gkn08R0AEltrs2orsbCb7JaKrUXcO5TuuwZlC8qySNd9dDrxgmxOyzVMttlmli8UVOIB0/MfOcpt3aeRbY6XuSa2KmseFAQdOCjh856P8Ay+P8jzWeHXMz/enmf9G/D4pfLn3/AL7eZVZCtZ9lIdWqa3uXc+Ou1WYtWTrqQeP9w6nXbOYwA3uB0Gv+/r9Jz+0rAdreHT/i8dTp5uCiv/3BpvdqQKxikcGsxhYw97G4/XenkebPL6e3+D5Lc2ULdmdYFblQN75Q9sx47LsS98oe2UNZKy8qZZa8i5rJWzylrNJQ9x8vrLmXPvzSCHtA5wd7yeXCVGNLmY5deW08bWPp68It70+vnKZFp68P3i3vTh+8iYoA4/eaODsW60Fgp3VDMTyGgGvP5QXE0V1Y6aL4jqNdemkOyNvWmvuaya6zqX0PjsJ9T6acNBALttYOPQqIlgsu13rNNdAu6fpx04c5iKNSB6x2PP5CToHn6cB7wAmx+SjgB+p9TDMbgpmfXxMPsOixNAFx1YxlEU2NgYW83eN+FOXUwDX2ZjjHpJbg7Dec+g9Jq9gaC/f5bc7W3U6Is5jtJneEVjnZz6LPRez2IKsalB+VBr7xs60iY2sYmR3ogmWkSZAnSMWgE96TDQfekg0A4wNJiyBd7GN05H0HWiLpL7RMs3yByIx1rfapFsuZByJA5EC61my+spfK6zKbIlbXwL5QdZkaOLAAw3SlqHk9ZBBB6aEj2JmHfXbiW130O6hXDY+QvBlYcd1tOTjzHn7GFm+XY2Wo1BIUNoHUqLKnAOoDIQQfpw8is1xrjj/I8Xz9z7eg7C+NyhAudjWGxQAbcYoRZ1KMRofYznu13xFpyLHtw8e2t7FUd7eyeBhw31rXUb2mnEtw56TNxOz2NeRuoOoozO71/tsrs3f+szYbs3gY6hrPsmOV47+XnNtK5v6cWlEB/u1HqDOjHl+HbnXHDr8fWvWo5Tsvs1mdb21AO/3JPNm5Nb/Sup4/zbo9dCu1G1RkZDMv8OtUppH+Gg5/Mlj85Lbe3K2DV43ebraCzIt3RbcBwACrwrQDgFHIcgOOvPlphf1V2eOTx55+60vIF5WWkGfSEideRYWlL3enGVu5Pt6SEuZc+vLb9HJ15xExhH10lMS0i19PrIkxQBRCIyVfP2BP6Rg2nDXqI0l+UdSYycxEFh/N8l/39JWI5PD3JJkYApb09P3kF4cfpJLA4JxV4y/LbyjYiyvIbUxKNi0F2Cjz/QTpnZaq90cAo1YwTZWN3ab7fiblM/bWVr92Dz4v/kIyoLvTberH81iAD0GoGk9roGiqPRR+08U2Wut9A/xa/wDyntfkPYRJSJkCfWMYxOsAWsixkdSOERPpAH3pEGR19JHX6wDz82SBtiinM93qDWyBsiijTagbZBrI8UaLVZeRLxoo+ItqJeQLxRSmdtQYyOseKNnaYmMTFFGi1Bn0lBMUUqMN32WsWkUUaCJjRRQBSWnLrrFFAGfmfcx18/aKKME/l7SMUUQKSQA66/KKKAIn/STQRRRKjRqGiyzZeOLLNTyU8oooG0do5G6Dw4IOAnM2OSSTzPOKKNI7s6uuXjj/ABFnspEaKIkDwkH4xRRhX3nkZFjpFFEDH1HCMeMUUA//2Q=='
test(url)
# test(url2)